In [33]:
from List_to_Pytorch_Dataset import dataset
import torch
from torch import nn
from torch.utils.data import DataLoader
import CNN1D

In [34]:
print(f"Dataset size: {len(dataset)}")

Dataset size: 2797


In [35]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda:0 device


In [36]:
labels = { 0  : "STD",
           1  : "WAL",
           2  : "JOG",
           3  : "JUM",
           4  : "STU",
           5  : "STN",
           6  : "SCH",
           7  : "SIT",
           8  : "CHU",
           9  : "CSI",
           10 : "CSO",
           11 : "FOL",
           12 : "FKL",
           13 : "BSC",
           14 : "SDL"}

classification_map = {}

for key, value in labels.items():
    classification_map[value] = {"TP" : 0,
                                 "FP" : 0,
                                 "TN" : 0,
                                 "FN" : 0}

In [37]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [38]:
def valid_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X)

            for key, value in labels.items():
                for i in range(len(y)):
                    if y[i] == key:
                        if pred[i].argmax() == key:
                            classification_map[value]["TP"] += 1
                        else:
                            classification_map[value]["FN"] += 1
                    elif pred[i].argmax() == key:
                        classification_map[value]["FP"] += 1
                    else:
                        classification_map[value]["TN"] += 1
                

            
            
            #model loss and accuracy
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    #model stats
    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n\n")

In [39]:
def main():
    model = CNN1D.CNN().to(device)
    model.train()

    learning_rate = 0.001
    epochs = 75

    # Initialize the loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #creating datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

    #Dataloaders for the datasets
    train_dataloader = DataLoader(train_set, batch_size = 64, shuffle = True)
    test_dataloader = DataLoader(test_set, batch_size = 64, shuffle = True)
    
    model.train()
    for _ in range(epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer)
    
    model.eval()

    valid_loop(test_dataloader, model, loss_fn)


In [40]:
main()

loss: 2.712485  [    0/ 2237]
loss: 2.186092  [  640/ 2237]
loss: 1.492962  [ 1280/ 2237]
loss: 1.306369  [ 1920/ 2237]
loss: 0.860147  [    0/ 2237]
loss: 1.197293  [  640/ 2237]
loss: 0.936832  [ 1280/ 2237]
loss: 0.638771  [ 1920/ 2237]
loss: 0.586583  [    0/ 2237]
loss: 0.495418  [  640/ 2237]
loss: 0.634991  [ 1280/ 2237]
loss: 0.637978  [ 1920/ 2237]
loss: 0.444129  [    0/ 2237]
loss: 0.476215  [  640/ 2237]
loss: 0.592631  [ 1280/ 2237]
loss: 0.428980  [ 1920/ 2237]
loss: 0.241496  [    0/ 2237]
loss: 0.262552  [  640/ 2237]
loss: 0.495441  [ 1280/ 2237]
loss: 0.445294  [ 1920/ 2237]
loss: 0.446908  [    0/ 2237]
loss: 0.336536  [  640/ 2237]
loss: 0.451514  [ 1280/ 2237]
loss: 0.296336  [ 1920/ 2237]
loss: 0.181393  [    0/ 2237]
loss: 0.186138  [  640/ 2237]
loss: 0.383402  [ 1280/ 2237]
loss: 0.198481  [ 1920/ 2237]
loss: 0.150080  [    0/ 2237]
loss: 0.241262  [  640/ 2237]
loss: 0.081042  [ 1280/ 2237]
loss: 0.233934  [ 1920/ 2237]
loss: 0.103547  [    0/ 2237]
loss: 0.07

In [45]:
stats_map = {}
for key, value in labels.items():
    stats_map[value] = {
        "Sensitivity" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Specificity" : float(classification_map[value]["TN"]) / float(classification_map[value]["TN"] + classification_map[value]["FP"]),
        "Recall" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FN"]),
        "Precision" : float(classification_map[value]["TP"]) / float(classification_map[value]["TP"] + classification_map[value]["FP"]),
        "Accuracy" : float(classification_map[value]["TP"] + classification_map[value]["TN"]) / float(classification_map[value]["TP"] + classification_map[value]["TN"] + classification_map[value]["FP"] + classification_map[value]["FN"])
    }
    stats_map[value]["F-score"] = 2.0 / float(1.0 / float(stats_map[value]["Precision"]) + 1.0 / float(stats_map[value]["Recall"]))
print(stats_map)

{'STD': {'Sensitivity': 0.9655172413793104, 'Specificity': 1.0, 'Recall': 0.9655172413793104, 'Precision': 1.0, 'Accuracy': 0.9982142857142857, 'F-score': 0.9824561403508772}, 'WAL': {'Sensitivity': 0.92, 'Specificity': 0.9921568627450981, 'Recall': 0.92, 'Precision': 0.92, 'Accuracy': 0.9857142857142858, 'F-score': 0.92}, 'JOG': {'Sensitivity': 0.96, 'Specificity': 0.9962616822429906, 'Recall': 0.96, 'Precision': 0.9230769230769231, 'Accuracy': 0.9946428571428572, 'F-score': 0.9411764705882353}, 'JUM': {'Sensitivity': 0.972972972972973, 'Specificity': 0.9904397705544933, 'Recall': 0.972972972972973, 'Precision': 0.8780487804878049, 'Accuracy': 0.9892857142857143, 'F-score': 0.9230769230769231}, 'STU': {'Sensitivity': 0.8571428571428571, 'Specificity': 0.9960861056751468, 'Recall': 0.8571428571428571, 'Precision': 0.9545454545454546, 'Accuracy': 0.9839285714285714, 'F-score': 0.9032258064516129}, 'STN': {'Sensitivity': 0.8717948717948718, 'Specificity': 0.9904030710172744, 'Recall': 0.